In [ ]:
try:
    h2o.remove_all()
    h2o.shutdown()
except:
    pass

In [ ]:
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
from collections import Counter
from itertools import chain
import h2o
import numpy as np
import pandas as pd
import seaborn as sns
from h2o.estimators.kmeans import H2OKMeansEstimator
from h2o.estimators.word2vec import H2OWord2vecEstimator
pd.options.display.max_columns= None
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
import nltk
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import matplotlib.pyplot as plt
import sys
import tensorflow as tf
np.set_printoptions(threshold=sys.maxsize)
from sklearn.preprocessing import MinMaxScaler
import gensim
from gensim.models import Word2Vec
from collections import Counter
from h2o.automl import H2OAutoML
import shap
import vaderSentiment

In [ ]:
#!pip install pandas_profiling --user --index-url=http://pypi.appdev.proj.coe.ic.gov/simple/ --trusted-host=pypi.appdev.proj.coe.ic.gov

In [ ]:
h2o.init()

## All Frames

In [ ]:
#join to get themes
categories=pd.read_excel("Theme_Key.xlsx")
data=pd.read_excel("All Comments.xls",sheets='All Comments')
data.reset_index()
to_w=data.melt(id_vars=['All Comments','RecNum'], value_vars=['Theme 1','Theme 2','Theme 3','Theme 4','Theme 5','Theme 6','Theme 7','Theme 8','Theme 9','Theme 10','Theme 11','Theme 12'])
to_w=to_w.merge(categories,on='value',how='inner')
to_w=to_w.drop(['variable','RecNum','value'],axis=1)
dat=to_w
data=dat
h2o_data=h2o.H2OFrame(data)
h2o_data=h2o_data.ascharacter()
h2o_data

In [ ]:
gl=h2o.import_file('glove.6B.300d.txt')

In [ ]:
#Sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
def sentiment_analyzer_scores(sentence):
    score=analyzer.polarity_scores(sentence)
    return(score)
Dat=data
J=Dat.dropna()['All Comments'].apply(str).apply(sentiment_analyzer_scores)

In [ ]:
#dynamiclly generate subsets
frames=[]
k=0
for i in dat['Theme'].unique():
    temp=dat
    temp2=temp[temp['Theme']==i]
    temp2['Bool']='True'
    temp3=temp[~temp.isin(temp2)].dropna()
    temp3['Bool']='False'
    temp2.reset_index(drop=True,inplace=True)
    temp3.reset_index(drop=True,inplace=True)
    temp=pd.concat([temp2,temp3],axis=0)

    frame_name=i.strip().replace(" ",'').replace("/",'').replace(",",'').replace("-",'')+'_FRAME'+'=temp'
    try:
        exec(frame_name)
    except:
        print(frame_name)
    frames.append(frame_name[:-5])
    k=k+1
    #if(k>0):
    #    break

frames

In [ ]:
frames

In [ ]:
#Pretrained with glove

STOP_WORDS = ["i","you","us","s","t","m","subject","can","lines","re","what",
               "there","all","we","one","the","a","an","of","or","in","for","by","on",
               "but","is","in","a","not","with","as","was","if","they","are","this","and","it","have",
               "from","at","my","be","by","not","that","to","from","com","org",
              "like","likes","so","(U)","(U//FOUO)","apos","quot","has","been","do","had","was","were",
             "they","their","them","am","will","than","when","who","where","our","me","your","would",'don','get','more',
             'nga','nro','nsa','new','adf','here','no','see','day','get','why','how','ve','dont','its','ive','work','people',
              'out','which','should','ADF-C','leadership','ADFC']


#This converts sentenses into vectors
def tokenize(sentences, stop_word = STOP_WORDS):
    tokenized = sentences.tokenize("\\W+")
    tokenized_lower = tokenized.tolower()
    tokenized_filtered = tokenized_lower[(tokenized_lower.nchar() >= 2) | (tokenized_lower.isna()),:]
    tokenized_words = tokenized_filtered[tokenized_filtered.grep("[0-9]",invert=True,output_logical=True),:]
    tokenized_words = tokenized_words[(tokenized_words.isna()) | (~ tokenized_words.isin(STOP_WORDS)),:]
    return tokenized_words
words=tokenize(h2o_data)

w2v_model = H2OWord2vecEstimator(pre_trained=gl)
w2v_model.train(training_frame=words)
survey= w2v_model.transform(words, aggregate_method = "AVERAGE")

In [ ]:
data['All Comments']
survey

In [ ]:
data

In [ ]:
%%capture output
#dynamically do all frames


h2o_all=h2o.H2OFrame(data).ascharacter()
#h2o_new=h2o.H2OFrame(data['All Comments'].ascharacter()) #this will be new data
words_all=tokenize(h2o_all)
#words_all=tokenize(h2o_new)



test_results=[]
survey_all= w2v_model.transform(words_all, aggregate_method = "AVERAGE")


k=0
for frame in frames:
    try:
        exec('h2o_data=h2o.H2OFrame('+frame+').ascharacter()')
        words=tokenize(h2o_data)
        survey= w2v_model.transform(words, aggregate_method = "AVERAGE")
        master=h2o_data.concat(survey)
        x=master.columns[2:]
        y='Bool'
        master['Bool']=master['Bool'].asfactor()
        master_train, test=master.split_frame(ratios=[.90])
        aml_frame='aml_'+frame
        exec(aml_frame+'=H2OAutoML(max_models=3,seed=1, balance_classes=True,nfolds=5,include_algos=[\'DeepLearning\',\'XGBoost\',\'StackedEnsemble\'])')
        exec(aml_frame+'.train(x=x,y=y, training_frame=master_train)')

        exec(aml_frame+"_model"+"="+aml_frame+".leader")

        
        exec('print('+aml_frame+'.leader.accuracy)')
        
        exec('pred_test='+aml_frame+'.leader.predict(test)')
        test_results.append(np.mean((pred_test['predict']==test['Bool']).as_data_frame()))
        exec('preds='+aml_frame+'.leader.predict(survey_all)') #Change to new data
        
        preds=preds.drop(1)
        col_names=[frame[:-5]+'_pred',frame[:-5]+'_true_prob']
        preds.set_names(col_names)
        h2o_all=preds.cbind( h2o_all)
    except:
        pass
    k=k+1
    #if(k>0):
    #    break

#h2o_all


In [ ]:
print("Models have average error-"+str(1-np.mean(test_results)))
print("With error bound-"+str(2*(np.var([1-x for x in test_results]))**.5/len(test_results)**.5))


In [ ]:
sns.distplot(test_results)

In [ ]:
test_results

In [ ]:
RedFlagcomments_FRAME

In [ ]:
len(test)

In [ ]:
#help(H2OAutoML)

In [ ]:
cell_out=str(output)
text_file=open("models.txt","w")
text_file.write(cell_out)
text_file.close()

In [ ]:
h2o.export_file(h2o_all,"debug.csv", force=True)

In [ ]:
h2o.export_file(h2o_all,"all_preds.csv",force=True)  

In [ ]:
change=pd.read_csv("all_preds.csv")
names=[col for col in change.columns if 'true' not in col]
d1=change[names]
d2=change['All Comments']
#d1.reset_index()
#d2.reset_index()
#d=pd.concat([d2,d1],axis=1)
d1=d1.drop(['Theme'],axis=1)

In [ ]:
import pandas_profiling as pf

In [ ]:
profile=pf.ProfileReport(d1)
profile.to_file(output_file='Report.html')

In [ ]:
d1